![title](http://i64.tinypic.com/157bpec.jpg)

With business data, we often have numerical, categorical data, missing data, outliers ... here's a short collection of tricks:

- When to use mean or median with imputation?
- How to detect outliers with Tukey's 1.5 IQR?
- Kaggle trick: how to label outliers.
- Kaggle trick: how to deal with MCAR / MAR at the same time?
- When to normalize or standardize?
- Should we care about cardinality and rare values?
- .....

# Loading the Data

In [39]:
import pandas as pd
import numpy as np

In [40]:
data = pd.read_excel("Panda_Master.xlsx")

In [41]:
data.head(32)

,Name,Age,Gender,Pre_Test_Score,Post_Test_Score,Country,State,Label
0,Jason,42,Male,101.0,103,USA,CA,0
1,Molly,52,Female,NaN,191,USA,MI,0
2,Tina,35,Female,121.0,115,USA,NY,1
3,Jake,24,Male,92.0,112,USA,OR,0
4,NaN,22,Male,98.0,134,USA,IL,0
5,Heidi,35,Female,95.0,101,Germany,HE,0
6,Susanne,34,Female,97.0,98,Germany,BA,0
7,Luisa,38,Female,101.0,100,USA,CA,0


In [42]:
data.shape

(8, 8)

# Check for NaN's

In [43]:
# Check for NaN's
data[['Name', 'Age', 'Pre_Test_Score', 'Post_Test_Score', 'Country']].isnull().sum()

Name               1
Age                0
Pre_Test_Score     1
Post_Test_Score    0
Country            0
dtype: int64

In [44]:
# MUCH BETTER ?????
# we have 2 NaN's
data.isnull().sum(axis=0)

Name               1
Age                0
Gender             0
Pre_Test_Score     1
Post_Test_Score    0
Country            0
State              0
Label              0
dtype: int64

In [45]:
data.loc[:,'Name'].head(3)

0    Jason
1    Molly
2     Tina
Name: Name, dtype: object

In [46]:
data.isnull().sum(axis=1)

0    0
1    1
2    0
3    0
4    1
5    0
6    0
7    0
dtype: int64

In [47]:
# let's look at the first NaN
data.iloc[1:2,:]

,Name,Age,Gender,Pre_Test_Score,Post_Test_Score,Country,State,Label
1,Molly,52,Female,NaN,191,USA,MI,0


# Outlier Detection
Detecting outliers is unfortunately more of an art than science. The famous statistician John Tukey proposed as IQR * 1.5 as a “outlier”. Hence, the upper fence is 75% + (IQR * 1.5).

In [48]:
# Max value
data['Pre_Test_Score'].max()

121.0

In [49]:
# Min value
data['Pre_Test_Score'].min()

92.0

In [50]:
# Calculating outlier values according to Tunkey's fence
IQR_2 = data.Pre_Test_Score.quantile(0.75) - data.Pre_Test_Score.quantile(0.25)

Lower_fence_2 = data.Pre_Test_Score.quantile(0.25) - (IQR_2 * 1.5)
Upper_fence_2 = data.Pre_Test_Score.quantile(0.75) + (IQR_2 * 1.5)

In [51]:
Upper_fence_2, Lower_fence_2, IQR_2

(108.5, 88.5, 5.0)

In [52]:
# How many outliers do we have?
data[data['Pre_Test_Score'] > 108.50].apply(lambda x: x.count())

Name               1
Age                1
Gender             1
Pre_Test_Score     1
Post_Test_Score    1
Country            1
State              1
Label              1
dtype: int64

Comment: We have 1 outlier out of 5

# Add a New Column for Outliers

In [53]:
data['outliers'] = np.where(data['Pre_Test_Score'] >= 108.50, 1,0)

In [54]:
data.head(4)

,Name,Age,Gender,Pre_Test_Score,Post_Test_Score,Country,State,Label,outliers
0,Jason,42,Male,101.0,103,USA,CA,0,0
1,Molly,52,Female,NaN,191,USA,MI,0,0
2,Tina,35,Female,121.0,115,USA,NY,1,1
3,Jake,24,Male,92.0,112,USA,OR,0,0


In [55]:
data.groupby('outliers').mean()

,Age,Pre_Test_Score,Post_Test_Score,Label
outliers,,,,
0,35.285714,97.333333,119.857143,0.0
1,35.000000,121.000000,115.000000,1.0


# Cardinality

In [56]:
# let's have a look at how many labels

for col in data[['Country', 'State']]:
    print(col, ': ', len(data[col].unique()), ' labels')

Country :  2  labels
State :  7  labels


In [57]:
data.groupby('Country').apply(lambda x: x.count())

,Name,Age,Gender,Pre_Test_Score,Post_Test_Score,Country,State,Label,outliers
Country,,,,,,,,,
Germany,2,2,2,2,2,2,2,2,2
USA,5,6,6,5,6,6,6,6,6


In [58]:
data['Country'].value_counts()

USA        6
Germany    2
Name: Country, dtype: int64

# NAN's: How to tackle MCAR + MAR at the same time?
If NAN is >5% of the dataset, interpolate NAN’s with mean (if data distribution is Gaussian) or median (if non-normal distribution). 

Additionally add a new column labeling NAN’s with 1, else 0. Thus covering 2 angles: 

(1) If the data was missing completely at random (MCAR), this would be contemplated by the mean imputation, and

(2) If it wasn't this would be captured by the additional variable.

In [64]:
# So we have a missting number in Pre_Test_Score
data.isnull().sum(axis=0)

Name               1
Age                0
Gender             0
Pre_Test_Score     1
Post_Test_Score    0
Country            0
State              0
Label              0
outliers           0
NAN_Labels         0
dtype: int64

In [72]:
# What's the mean of Pre_Test_Score?
data['Pre_Test_Score'].mean()

100.71375

In [74]:
data['Pre_Test_Score'].fillna(value=100.71, inplace=True)

In [75]:
data['NAN_Labels'] = np.where(data['Pre_Test_Score'] == 100.71, 1,0)

In [76]:
data.head()

,Name,Age,Gender,Pre_Test_Score,Post_Test_Score,Country,State,Label,outliers,NAN_Labels
0,Jason,42,Male,101.00,103,USA,CA,0,0,0
1,Molly,52,Female,100.71,191,USA,MI,0,0,1
2,Tina,35,Female,121.00,115,USA,NY,1,1,0
3,Jake,24,Male,92.00,112,USA,OR,0,0,0
4,NaN,22,Male,98.00,134,USA,IL,0,0,0


# Dealing with a Missing Name

In [33]:
# Missing name?
data[['Name']].isnull().sum()

Name    1
dtype: int64

In [34]:
# What are the names?
data['Name'].value_counts()

Jason      1
Jake       1
Susanne    1
Tina       1
Heidi      1
Molly      1
Luisa      1
Name: Name, dtype: int64

In [35]:
# Replace the missing name with 'Unknown'
data['Name'].fillna(value='Unknown', inplace=True)

# Dealing with a missing Country name

In [224]:
# Replace missing country with most common country
data['Country'].fillna(value='USA', inplace=True)

# Encoding Categorical Labels
Categorical information is often represented in data as a vector or column of strings (e.g. "California"). The problem is that most machine learning algorithms require inputs to be numerical values.

# Gender Encoding with map()

In [225]:
# Let's encode 'Gender' with Python's map() function.

In [226]:
Gender_ = data['Gender'].map({'Female': 0, 'Male':1})
Gender_.head(2)

0    1
1    0
Name: Gender, dtype: int64

# Country Encoding with LabelEncoder()

In [227]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [228]:
le.fit(data['Country'])

LabelEncoder()

In [229]:
Country_ = le.transform(data['Country'])

In [230]:
Country_

array([1, 1, 1, 1, 1, 0, 0, 1])

# Name Encoding with LabelEncoder()

In [231]:
le.fit(data['Name'])

LabelEncoder()

In [232]:
Name_ = le.transform(data['Name'])

In [233]:
Name_

array([2, 4, 6, 1, 7, 0, 5, 3])

# State Encoding with LabelEncoder()

In [234]:
le.fit(data['State'])

LabelEncoder()

In [235]:
State_ = le.transform(data['State'])

In [236]:
State_

array([1, 4, 5, 6, 3, 2, 0, 1])